In [1]:
#imports
import pandas as pd
import numpy as np
import os
import folium
from folium import IFrame
import json
from folium import plugins
from scipy.ndimage import imread
import random
import copy
import vega
import vincent
import collections
from collections import Counter
from collections import defaultdict
from itertools import filterfalse
import time
import csv
import ast
import re
import base64
from PIL import Image, ImageTk

In [2]:
vincent.core.initialize_notebook()

In [4]:
json_path = r'data/map_data/world-countries.json'

world_countries = json.load(open(json_path))

map_data = folium.Map(location=[46.947456, 7.451123], tiles='Mapbox Bright', zoom_start=2)

style_function = lambda x: {'weight': '0.4',
                            'fillColor': '#FF0000' if
                                x['properties']['name']=='Switzerland' else
                                         '#34BED2'}

geojson = folium.GeoJson(
    world_countries,
    name='geojson',
    style_function=style_function,
).add_to(map_data)
popup = folium.Popup("a country")
popup.add_to(geojson)

folium.LayerControl().add_to(map_data)
map_data

In [6]:
db = pd.read_csv('cleaning_data/categories_db.csv')

In [7]:
dict_to_plot = {'products': db.Count.tolist(), 'amount': db.category_main.tolist()}

In [9]:
index_loc = range(db.shape[1])
categories = db.category_main.values.tolist()
counts = db.Count.values.tolist()
zipped = zip(categories, counts)
dic_db = dict(list(zipped))
dic_db

{'Dried fruits and nuts': 3,
 'Fruits': 6,
 'Vegetables': 6,
 'Condiments': 7,
 'Honeys': 10,
 'Oils': 14,
 'Fermented products': 19,
 'Tee and infusions': 20,
 'Baby food': 22,
 'Canned products': 24,
 'Spreads': 34,
 'Sauces': 37,
 'Frozen products': 43,
 'Alcoholic beverages': 49,
 'Confections': 78,
 'Ready meals': 101,
 'Drinks': 134,
 'Grains, beans and legumes': 135,
 'Meat, fish and egg': 163,
 'Plant based products': 194,
 'Dairy products': 337}

In [10]:
from vincent import *

bar = vincent.Bar(dic_db)
bar.axis_titles(x="", y="Quantity")
bar.height = 150 
bar.width = 300
ax = AxisProperties(labels = PropertySet(angle=ValueRef(value=45), align=ValueRef(value="left")))
bar.axes[0].properties = ax
 
bar.to_json('barplot.json')
bar

In [14]:
json_path = r'data/map_data/world-countries.json'
barplot = json.load(open(r'barplot.json'))
world_countries = json.load(open(json_path))
map_data = folium.Map(location=[46.947456, 7.451123], tiles='Mapbox Bright', zoom_start=5, max_zoom=7, min_zoom=3)

style_function = lambda x: {'weight': '0.4',
                            'fillColor': '#FF0000' if
                                x['properties']['name']=='Switzerland' else
                                         '#34BED2'}

for feature in world_countries["features"] :
    coord = copy.deepcopy(feature['geometry']['coordinates'])
    popup_text = "<b>Country name</b>: "+ copy.deepcopy(feature['properties']['name']+"<br>")
    
    if feature['properties']['name'] == 'Switzerland':
        popup = folium.Popup(popup_text, max_width=400) 
        folium.Vega(bar, height=300, width=1000).add_to(popup)
    else:
        popup = folium.Popup(popup_text)
        
    geojson = folium.GeoJson(feature,
                    style_function=style_function)
    popup.add_to(geojson)
    geojson.add_to(map_data)
map_data.save('docs/img/maps/swiss_map.html')
map_data